In [38]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor, AutoModel, AutoModelForImageClassification

# path = '/workspace/Weight_compression/Wparam_dataset/hf_model/llava-hf--llava-1.5-7b-hf'

# model = LlavaForConditionalGeneration.from_pretrained(path)

In [41]:
model = AutoModelForImageClassification.from_pretrained("/workspace/Weight_compression/Wparam_dataset/hf_model/facebook--dinov2-large-imagenet1k-1-layer")

In [40]:
model

Dinov2ForImageClassification(
  (dinov2): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-23): 24 x Dinov2Layer(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2SdpaAttention(
            (attention): Dinov2SdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inpl

In [ ]:
import datasets
from datasets import load_dataset
import os
os.environ['HF_HOME'] = "/workspace/hf_cache/huggingface_nwc"

dataset = load_dataset("liuhaotian/LLaVA-Instruct-150K", split="train")


In [ ]:
import pandas as pd
from datasets.arrow_dataset import Dataset as HFDataset
# df = pd.read_json("/workspace/hf_cache/huggingface_nwc/hub/datasets--liuhaotian--LLaVA-Instruct-150K/snapshots/9d451dc7629cfe0469f6ae4432b765cd603d5fcb/llava_v1_5_mix665k.json").drop(columns=["id", "model"])
path = '/workspace/hf_cache/huggingface_nwc/hub/datasets--liuhaotian--LLaVA-Instruct-150K/snapshots/9d451dc7629cfe0469f6ae4432b765cd603d5fcb/llava_instruct_150k.json'
df = pd.read_json(path).drop(columns=["id"])

ds = HFDataset.from_pandas(df, preserve_index=False)
    # llava_features = Features({
    #         'id': Value('string'),
    #         'image': Image(),
    #         'conversations': Sequence({
    #             'from': Value('string'),
    #             'value': Value('string'),
    #         })
    #     })
    # print("loading dataset with explicit features...")
    # load_dataset 호출 시 'features' 인자를 추가합니다.
    # dataset = load_dataset("liuhaotian/LLaVA-Instruct-150K", split="train", features=llava_features)
    # data_files = {"train": "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json"}
    # dataset = load_dataset("json", data_files=data_files, split="train", features=llava_features)
    # dataset = load_dataset("liuhaotian/LLaVA-Instruct-150K", split="train")

In [ ]:
ds = ds.shuffle(seed=42).select(range(10))

In [ ]:
ds['image']

In [26]:
import torch
q = torch.load('/workspace/Weight_compression/Wparam_dataset/quip_hess/llava-hf--llava-1.5-7b-hf_512/lang_0_q.pt')
v = torch.load('/workspace/Weight_compression/Wparam_dataset/quip_hess/llava-hf--llava-1.5-7b-hf_512/lang_0_k.pt')

In [21]:
q

{'n': 4096,
 'ct': 2097152,
 'flatH': tensor([ 3.3080e-04,  9.9871e-05,  7.8034e-05,  ..., -3.4949e-07,
          3.6878e-06,  7.5221e-06])}

In [22]:
k

{'n': 4096,
 'ct': 2097152,
 'flatH': tensor([ 3.3080e-04,  9.9871e-05,  7.8034e-05,  ..., -3.4949e-07,
          3.6878e-06,  7.5221e-06])}

In [27]:
def compare_dicts(d1, d2, tol=1e-8):
    if d1.keys() != d2.keys():
        return False
    for k in d1:
        v1, v2 = d1[k], d2[k]
        if isinstance(v1, torch.Tensor) and isinstance(v2, torch.Tensor):
            # if not torch.allclose(v1, v2, atol=tol, rtol=0):
            if not torch.equal(v1, v2):
                return False
        else:
            if v1 != v2:
                return False
    return True

print(compare_dicts(q, v))  

True
